In [ ]:
import json
import os
from typing import Dict, List, Any, Optional
from collections import defaultdict

def aplicar_mapeo_logos():
    """
    Aplica el mapeo de nombres de logos al archivo JSON unificado del concurso fAIcing.
    Maneja duplicados, archivos faltantes y genera reportes detallados.
    """
    
    # Mapeo de nombres proporcionado
    mapping_data = {
        "mapping": {
            "logo_01.png": ["tanda_1_logo_01.png"],
            "logo_02.png": ["tanda_1_logo_02.png"],
            "logo_03.png": ["tanda_1_logo_03.png"],
            "logo_04.png": ["tanda_1_logo_04.png"],
            "logo_05.png": ["tanda_1_logo_05.png"],
            "logo_06.png": ["tanda_1_logo_06.png"],
            "logo_07.png": ["tanda_1_logo_07.png"],
            "logo_08.png": ["tanda_1_logo_08.png"],
            "logo_09.png": ["tanda_1_logo_09.png", "tanda_3_logo_08.png"],
            "logo_10.png": ["tanda_1_logo_10.png"],
            "logo_11.png": ["tanda_4_logo_08.png"],
            "logo_12.png": ["tanda_2_logo_03.png"],
            "logo_13.png": ["tanda_2_logo_01.png"],
            "logo_14.png": ["tanda_2_logo_04.png"],
            "logo_17.png": ["tanda_3_logo_04.png"],
            "logo_18.png": ["tanda_4_logo_10.png"],
            "logo_19.png": ["tanda_2_logo_10.png", "tanda_4_logo_24.png"],
            "logo_20.png": ["tanda_2_logo_05.png"],
            "logo_21.png": ["tanda_4_logo_11.png"],
            "logo_23.png": ["tanda_3_logo_03.png"],
            "logo_25.png": ["tanda_2_logo_02.png"],
            "logo_27.png": ["tanda_3_logo_06.png"],
            "logo_28.png": ["tanda_4_logo_13.png"],
            "logo_30.png": ["tanda_3_logo_09.png"],
            "logo_31.png": ["tanda_3_logo_10.png"],
            "logo_35.png": ["tanda_4_logo_03.png"],
            "logo_36.png": ["tanda_2_logo_06.png", "tanda_4_logo_19.png"],
            "logo_37.png": ["tanda_4_logo_02.png"],
            "logo_38.png": ["tanda_4_logo_18.png"],
            "logo_40.png": ["tanda_4_logo_05.png"],
            "logo_41.png": ["tanda_3_logo_07.png"],
            "logo_43.png": ["tanda_4_logo_06.png"],
            "logo_44.png": ["tanda_4_logo_09.png"],
            "logo_46.png": ["tanda_3_logo_02.png"],
            "logo_47.png": ["tanda_3_logo_05.png"],
            "logo_48.png": ["tanda_2_logo_09.png"],
            "logo_49.png": ["tanda_2_logo_07.png"],
            "logo_50.png": ["tanda_2_logo_08.png", "tanda_4_logo_14.png"],
            "logo_51.png": ["tanda_4_logo_15.png"],
            "logo_52.png": ["tanda_4_logo_01.png"],
            "logo_53.png": ["tanda_4_logo_07.png", "tanda_4_logo_17.png"],
            "logo_55.png": ["tanda_4_logo_04.png"],
            "logo_57.png": ["tanda_4_logo_16.png", "tanda_4_logo_20.png"],
            "logo_58.png": ["tanda_4_logo_21.png"],
            "logo_59.png": ["tanda_3_logo_01.png", "tanda_4_logo_22.png"],
            "logo_60.png": ["tanda_4_logo_23.png"],
            "logo_61.png": ["tanda_4_logo_12.png"]
        },
        "unsubmitted_or_skipped_files": [
            "logo_15.png", "logo_16.png", "logo_22.png", "logo_24.png",
            "logo_26.png", "logo_29.png", "logo_32.png", "logo_33.png",
            "logo_34.png", "logo_39.png", "logo_42.png", "logo_45.png",
            "logo_54.png", "logo_56.png"
        ]
    }
    
    # Configuración de archivos
    archivo_entrada = "concurso_logos_faicing_unificado.json"
    archivo_salida = "concurso_logos_faicing_mapeado.json"
    archivo_reporte = "reporte_mapeo_logos.json"
    
    # Validar que existe el archivo de entrada
    if not os.path.exists(archivo_entrada):
        print(f"❌ Error: No se encontró el archivo {archivo_entrada}")
        print("   Ejecuta primero el script unificador para generar este archivo.")
        return
    
    try:
        # Cargar datos unificados
        with open(archivo_entrada, 'r', encoding='utf-8') as f:
            datos_originales = json.load(f)
        
        print(f"✅ Archivo cargado: {len(datos_originales.get('logos', []))} logos encontrados")
        
        # Crear mapeo inverso para búsqueda eficiente
        mapeo_inverso = {}
        for nuevo_nombre, archivos_originales in mapping_data["mapping"].items():
            for archivo_original in archivos_originales:
                if archivo_original in mapeo_inverso:
                    print(f"⚠️  Advertencia: {archivo_original} mapea a múltiples logos nuevos")
                mapeo_inverso[archivo_original] = nuevo_nombre
        
        # Estadísticas de mapeo
        estadisticas_mapeo = {
            "logos_procesados": 0,
            "logos_mapeados": 0,
            "logos_sin_mapeo": 0,
            "logos_duplicados": 0,
            "mapeos_multiples": 0,
            "archivos_no_encontrados": [],
            "mapeos_aplicados": {},
            "logos_consolidados": {},
            "conflictos_resueltos": []
        }
        
        # Procesar logos
        logos_mapeados = []
        logos_no_mapeados = []
        mapeos_multiples = defaultdict(list)
        
        for logo in datos_originales.get("logos", []):
            estadisticas_mapeo["logos_procesados"] += 1
            logo_id_original = logo.get("logo_id", "")
            
            if logo_id_original in mapeo_inverso:
                nuevo_nombre = mapeo_inverso[logo_id_original]
                
                # Crear copia del logo con nuevo nombre
                logo_mapeado = logo.copy()
                logo_mapeado["logo_id"] = nuevo_nombre
                logo_mapeado["logo_id_original"] = logo_id_original
                logo_mapeado["mapeo_aplicado"] = True
                logo_mapeado["fecha_mapeo"] = "2025-01-01"
                
                # Verificar si ya existe este nuevo nombre
                if nuevo_nombre in [l.get("logo_id") for l in logos_mapeados]:
                    estadisticas_mapeo["logos_duplicados"] += 1
                    mapeos_multiples[nuevo_nombre].append(logo_mapeado)
                    print(f"🔄 Duplicado detectado: {nuevo_nombre} (de {logo_id_original})")
                else:
                    logos_mapeados.append(logo_mapeado)
                    estadisticas_mapeo["logos_mapeados"] += 1
                    estadisticas_mapeo["mapeos_aplicados"][logo_id_original] = nuevo_nombre
                
            else:
                # Logo sin mapeo
                logo_sin_mapeo = logo.copy()
                logo_sin_mapeo["mapeo_aplicado"] = False
                logo_sin_mapeo["razon_sin_mapeo"] = "No encontrado en mapping proporcionado"
                logos_no_mapeados.append(logo_sin_mapeo)
                estadisticas_mapeo["logos_sin_mapeo"] += 1
                estadisticas_mapeo["archivos_no_encontrados"].append(logo_id_original)
        
        # Resolver conflictos de mapeos múltiples
        for nuevo_nombre, logos_conflicto in mapeos_multiples.items():
            if len(logos_conflicto) > 1:
                estadisticas_mapeo["mapeos_multiples"] += 1
                
                # Estrategia de resolución: seleccionar el de mayor puntuación
                mejor_logo = max(logos_conflicto, key=lambda x: x.get("total", 0))
                logos_descartados = [l for l in logos_conflicto if l != mejor_logo]
                
                # Consolidar información de logos descartados
                mejor_logo["logos_consolidados"] = []
                for logo_descartado in logos_descartados:
                    mejor_logo["logos_consolidados"].append({
                        "logo_id_original": logo_descartado.get("logo_id_original"),
                        "tanda": logo_descartado.get("tanda"),
                        "total": logo_descartado.get("total"),
                        "razon_descarte": "Puntuación menor en mapeo múltiple"
                    })
                
                # Agregar metadatos de consolidación
                mejor_logo["es_consolidacion"] = True
                mejor_logo["cantidad_logos_consolidados"] = len(logos_descartados)
                mejor_logo["criterio_seleccion"] = "Mayor puntuación total"
                
                logos_mapeados.append(mejor_logo)
                
                # Registrar conflicto resuelto
                estadisticas_mapeo["conflictos_resueltos"].append({
                    "logo_final": nuevo_nombre,
                    "logo_seleccionado": mejor_logo.get("logo_id_original"),
                    "puntuacion_seleccionada": mejor_logo.get("total"),
                    "logos_descartados": [l.get("logo_id_original") for l in logos_descartados],
                    "puntuaciones_descartadas": [l.get("total") for l in logos_descartados]
                })
                
                print(f"🎯 Conflicto resuelto: {nuevo_nombre} → {mejor_logo.get('logo_id_original')} (puntuación: {mejor_logo.get('total')})")
        
        # Verificar archivos del mapeo que no se encontraron en los datos
        archivos_esperados = set()
        for archivos_lista in mapping_data["mapping"].values():
            archivos_esperados.update(archivos_lista)
        
        archivos_encontrados = set(logo.get("logo_id", "") for logo in datos_originales.get("logos", []))
        archivos_faltantes = archivos_esperados - archivos_encontrados
        
        if archivos_faltantes:
            print(f"⚠️  Archivos esperados pero no encontrados: {len(archivos_faltantes)}")
            for archivo in sorted(archivos_faltantes):
                print(f"   - {archivo}")
        
        # Reordenar logos mapeados por número de logo
        def extraer_numero_logo(logo_id):
            try:
                # Extraer número de "logo_XX.png"
                numero = logo_id.replace("logo_", "").replace(".png", "")
                return int(numero)
            except (ValueError, AttributeError):
                return 999999  # Poner al final si no se puede parsear
        
        logos_mapeados.sort(key=lambda x: extraer_numero_logo(x.get("logo_id", "")))
        
        # Recalcular rankings globales
        todos_los_logos = logos_mapeados + logos_no_mapeados
        todos_los_logos.sort(key=lambda x: x.get("total", 0), reverse=True)
        
        for idx, logo in enumerate(todos_los_logos, 1):
            logo["ranking_global"] = idx
        
        # Actualizar estadísticas generales
        estadisticas_actualizadas = datos_originales.get("estadisticas", {}).copy()
        estadisticas_actualizadas.update({
            "total_logos_mapeados": len(logos_mapeados),
            "total_logos_sin_mapeo": len(logos_no_mapeados),
            "archivos_consolidados": estadisticas_mapeo["mapeos_multiples"],
            "fecha_ultimo_mapeo": "2025-01-01"
        })
        
        # Estructura final de datos mapeados
        datos_mapeados = {
            "metadata": {
                **datos_originales.get("metadata", {}),
                "version": "2.0",
                "descripcion": "Logos del concurso fAIcing con mapeo de nombres aplicado",
                "mapeo_aplicado": True,
                "fecha_mapeo": "2025-01-01"
            },
            "estadisticas": estadisticas_actualizadas,
            "estadisticas_mapeo": estadisticas_mapeo,
            "criterios_evaluacion": datos_originales.get("criterios_evaluacion", {}),
            "mapeo_utilizado": mapping_data,
            "logos": todos_los_logos,
            "logos_por_categoria": {
                "mapeados": logos_mapeados,
                "sin_mapeo": logos_no_mapeados
            }
        }
        
        # Guardar archivo mapeado
        with open(archivo_salida, 'w', encoding='utf-8') as f:
            json.dump(datos_mapeados, f, ensure_ascii=False, indent=2)
        
        # Crear reporte detallado
        reporte_detallado = {
            "resumen_mapeo": {
                "fecha_procesamiento": "2025-01-01",
                "archivo_origen": archivo_entrada,
                "archivo_destino": archivo_salida,
                "total_mapeos_definidos": len(mapping_data["mapping"]),
                "total_logos_procesados": estadisticas_mapeo["logos_procesados"],
                "total_logos_mapeados": estadisticas_mapeo["logos_mapeados"],
                "total_conflictos_resueltos": len(estadisticas_mapeo["conflictos_resueltos"]),
                "archivos_no_enviados": len(mapping_data["unsubmitted_or_skipped_files"])
            },
            "estadisticas_detalladas": estadisticas_mapeo,
            "mapeos_aplicados": {
                "exitosos": estadisticas_mapeo["mapeos_aplicados"],
                "conflictos_resueltos": estadisticas_mapeo["conflictos_resueltos"]
            },
            "archivos_faltantes": list(archivos_faltantes),
            "archivos_no_enviados": mapping_data["unsubmitted_or_skipped_files"],
            "validacion": {
                "archivos_esperados": len(archivos_esperados),
                "archivos_encontrados": len(archivos_encontrados),
                "archivos_faltantes": len(archivos_faltantes),
                "integridad_ok": len(archivos_faltantes) == 0
            }
        }
        
        # Guardar reporte
        with open(archivo_reporte, 'w', encoding='utf-8') as f:
            json.dump(reporte_detallado, f, ensure_ascii=False, indent=2)
        
        # Reporte final en consola
        print("\n" + "="*80)
        print("🎯 MAPEO DE LOGOS COMPLETADO - CONCURSO fAIcing")
        print("="*80)
        print(f"📁 Archivo procesado: {archivo_entrada}")
        print(f"💾 Archivo generado: {archivo_salida}")
        print(f"📊 Reporte detallado: {archivo_reporte}")
        print()
        print("📈 ESTADÍSTICAS DE MAPEO:")
        print(f"   • Logos procesados: {estadisticas_mapeo['logos_procesados']}")
        print(f"   • Logos mapeados exitosamente: {estadisticas_mapeo['logos_mapeados']}")
        print(f"   • Logos sin mapeo: {estadisticas_mapeo['logos_sin_mapeo']}")
        print(f"   • Conflictos resueltos: {len(estadisticas_mapeo['conflictos_resueltos'])}")
        print(f"   • Archivos no enviados: {len(mapping_data['unsubmitted_or_skipped_files'])}")
        print()
        
        if estadisticas_mapeo["conflictos_resueltos"]:
            print("🔄 CONFLICTOS RESUELTOS:")
            for conflicto in estadisticas_mapeo["conflictos_resueltos"]:
                print(f"   • {conflicto['logo_final']}: {conflicto['logo_seleccionado']} "
                      f"(puntuación {conflicto['puntuacion_seleccionada']}) seleccionado sobre "
                      f"{', '.join(conflicto['logos_descartados'])}")
        
        if archivos_faltantes:
            print(f"\n⚠️  ARCHIVOS FALTANTES ({len(archivos_faltantes)}):")
            for archivo in sorted(archivos_faltantes)[:10]:  # Mostrar solo los primeros 10
                print(f"   • {archivo}")
            if len(archivos_faltantes) > 10:
                print(f"   ... y {len(archivos_faltantes) - 10} más")
        
        print(f"\n✅ Mapeo completado exitosamente")
        print(f"🏆 Top 3 logos después del mapeo:")
        for i, logo in enumerate(todos_los_logos[:3], 1):
            emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
            print(f"   {emoji} {logo.get('logo_id')} (Tanda {logo.get('tanda')}) - {logo.get('total')}/60 pts")
        
        return datos_mapeados
        
    except json.JSONDecodeError as e:
        print(f"❌ Error al leer el archivo JSON: {e}")
    except Exception as e:
        print(f"❌ Error durante el procesamiento: {e}")
        import traceback
        traceback.print_exc()

def validar_integridad_mapeo():
    """
    Función adicional para validar la integridad del mapeo aplicado
    """
    archivo_mapeado = "concurso_logos_faicing_mapeado.json"
    
    if not os.path.exists(archivo_mapeado):
        print("❌ Archivo mapeado no encontrado. Ejecuta primero aplicar_mapeo_logos()")
        return
    
    try:
        with open(archivo_mapeado, 'r', encoding='utf-8') as f:
            datos = json.load(f)
        
        logos = datos.get("logos", [])
        mapeo = datos.get("mapeo_utilizado", {}).get("mapping", {})
        
        print("\n🔍 VALIDACIÓN DE INTEGRIDAD DEL MAPEO")
        print("="*50)
        
        # Verificar que todos los logos mapeados tienen nombres correctos
        logos_con_formato_correcto = 0
        logos_con_formato_incorrecto = []
        
        for logo in logos:
            logo_id = logo.get("logo_id", "")
            if logo.get("mapeo_aplicado"):
                if logo_id.startswith("logo_") and logo_id.endswith(".png"):
                    logos_con_formato_correcto += 1
                else:
                    logos_con_formato_incorrecto.append(logo_id)
        
        print(f"✅ Logos con formato correcto: {logos_con_formato_correcto}")
        if logos_con_formato_incorrecto:
            print(f"❌ Logos con formato incorrecto: {len(logos_con_formato_incorrecto)}")
            for logo_id in logos_con_formato_incorrecto[:5]:
                print(f"   • {logo_id}")
        
        # Verificar duplicados
        logo_ids = [logo.get("logo_id") for logo in logos]
        duplicados = [logo_id for logo_id in set(logo_ids) if logo_ids.count(logo_id) > 1]
        
        if duplicados:
            print(f"⚠️  Duplicados encontrados: {duplicados}")
        else:
            print("✅ No se encontraron duplicados")
        
        # Verificar cobertura del mapeo
        logos_mapeados = sum(1 for logo in logos if logo.get("mapeo_aplicado"))
        logos_esperados = len(mapeo)
        
        print(f"📊 Cobertura: {logos_mapeados}/{logos_esperados} logos mapeados")
        
        estadisticas = datos.get("estadisticas_mapeo", {})
        print(f"🔄 Conflictos resueltos: {len(estadisticas.get('conflictos_resueltos', []))}")
        
        print("\n✅ Validación completada")
        
    except Exception as e:
        print(f"❌ Error durante la validación: {e}")

print("🚀 Iniciando mapeo de logos del concurso fAIcing...")
resultado = aplicar_mapeo_logos()

if resultado:
    print("\n🔍 Ejecutando validación de integridad...")
    validar_integridad_mapeo()